In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import pandas as pd
import medmnist
from medmnist import INFO, Evaluator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from scipy import stats
from PIL import Image
import pdb
import math

/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/grads/a/akshatha.mohan/anaconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Example usage:
image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)
image


tensor([[[[ 9., 11.,  8., 20.],
          [18., 18., 17.,  3.],
          [10.,  7., 18., 16.],
          [18., 17., 22., 13.]]]])

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import math
import pdb

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        windows_horizontal = []
        Lr_all_horizontal = []

        for i in range(image_scaled.size(2) - self.window_size + 1):
            for j in range(image_scaled.size(3) - self.window_size + 1):
                window = image_scaled[:, :, i:i+self.window_size, j:j+self.window_size]
                windows_horizontal.append(window)

                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(window)
                min_pool_output = -max_pool(-window)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

        return L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)
image = torch.rand((128, 3, 13, 13), dtype=torch.float32)
# Assuming 'your_image_tensor' is your input tensor
result = custom_pooling_layer(image)
print(result)


tensor([[[[0.0139]],

         [[0.0169]],

         [[0.0132]]],


        [[[0.0145]],

         [[0.0128]],

         [[0.0130]]],


        [[[0.0123]],

         [[0.0130]],

         [[0.0172]]],


        [[[0.0167]],

         [[0.0137]],

         [[0.0169]]],


        [[[0.0127]],

         [[0.0147]],

         [[0.0135]]],


        [[[0.0133]],

         [[0.0152]],

         [[0.0132]]],


        [[[0.0130]],

         [[0.0143]],

         [[0.0130]]],


        [[[0.0161]],

         [[0.0147]],

         [[0.0147]]],


        [[[0.0143]],

         [[0.0227]],

         [[0.0149]]],


        [[[0.0149]],

         [[0.0133]],

         [[0.0169]]],


        [[[0.0145]],

         [[0.0137]],

         [[0.0137]]],


        [[[0.0132]],

         [[0.0125]],

         [[0.0123]]],


        [[[0.0167]],

         [[0.0145]],

         [[0.0130]]],


        [[[0.0172]],

         [[0.0185]],

         [[0.0145]]],


        [[[0.0167]],

         [[0.0175]],

    

In [36]:
import torch
import torch.nn as nn
import math

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        windows_horizontal = image.unfold(2, self.window_size, 1).unfold(3, self.window_size, 1)
        windows_horizontal = windows_horizontal.squeeze(0).squeeze(0)
        print(windows_horizontal.shape)
        max_pool = nn.MaxPool2d(kernel_size=self.window_size)
        max_pool_output = max_pool(windows_horizontal)
        min_pool_output = -max_pool(-windows_horizontal)

        nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
        Mr = torch.sum(nr)
        Q_mr = nr / (self.window_size - self.r + 1)
        L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

        return L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor
image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)

# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result)


torch.Size([2, 2, 3, 3])
tensor([[[[0.5000]],

         [[0.2000]]],


        [[[0.2500]],

         [[0.1667]]]])


Batch dimension stack

In [74]:
import torch
import torch.nn as nn

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []

        # Iterate over batches
        for batch in range(batch_size):
            # Extract the current batch's unfolded window
            windows_horizontal = image[batch].unfold(1, self.window_size, 1).unfold(2, self.window_size, 1)
            windows_horizontal = windows_horizontal.squeeze(0)

            # Perform operations independently for each window in the current batch
            max_pool = nn.MaxPool2d(kernel_size=self.window_size)
            max_pool_output = max_pool(windows_horizontal)
            min_pool_output = -max_pool(-windows_horizontal)

            nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
            Mr = torch.sum(nr)
            Q_mr = nr / (self.window_size - self.r + 1)
            L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

            # Accumulate the result for the current batch
            all_L_r.append(L_r)

        # Concatenate results along the batch dimension
        all_L_r = torch.cat(all_L_r, dim=0)

        return all_L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)


image = torch.tensor([[9, 11, 8, 20], [18, 18, 17, 3], [10, 7, 18, 16], [18, 17, 22, 13]], dtype=torch.float32)
image = image.unsqueeze(0).unsqueeze(0)

# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result.shape)


torch.Size([2, 2, 1, 1])


Channel dimension stack

In [38]:
import torch
import torch.nn as nn

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        # Scale the input tensor to the range [0, 255]
        image_scaled = image * 255.0

        windows_horizontal = image_scaled.unfold(2, self.window_size, 1).unfold(3, self.window_size, 1)
        windows_horizontal = windows_horizontal.squeeze(0).squeeze(0)

        max_pool = nn.MaxPool2d(kernel_size=self.window_size)
        max_pool_output = max_pool(windows_horizontal)
        min_pool_output = -max_pool(-windows_horizontal)


        nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
        Mr = torch.sum(nr)
        Q_mr = nr / (self.window_size - self.r + 1)
        L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

        return L_r

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor with shape (1, 1, 4, 4)
image = torch.rand((1, 1, 4, 4), dtype=torch.float32)

# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result.shape)


torch.Size([2, 2, 1, 1])


Normalized dimension stack

In [17]:
import torch
import torch.nn as nn

class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []
        all_L_r_channel = []

        # Iterate over batches
        for batch in range(batch_size):
            # Iterate over channels
            channel_L_r = []
            for channel in range(channels):
                # Extract the current batch's unfolded window for the current channel
                windows_horizontal = image_scaled[batch, channel].unfold(0, self.window_size, 1).unfold(1, self.window_size, 1)
                windows_horizontal = windows_horizontal.squeeze(0)

                # Perform operations independently for each window in the current channel
                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(windows_horizontal)
                min_pool_output = -max_pool(-windows_horizontal)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2
                # Accumulate the result for the current channel
                channel_L_r.append(L_r)

            # Concatenate results along the channel dimension for the current batch
            channel_L_r = torch.cat(channel_L_r, dim=0)
            all_L_r_channel.append(channel_L_r)

        # Concatenate results along the batch dimension
        all_L_r_batch = torch.cat(all_L_r_channel, dim=1)

        return all_L_r_batch

# Example usage:
# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor with shape [128, 1, 13, 13] (128 batches, 1 channel, 13x13 image)
image = torch.rand((128, 3, 13, 13), dtype=torch.float32)


# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result)


tensor([[[[0.0208]],

         [[0.0145]],

         [[0.0145]],

         ...,

         [[0.0169]],

         [[0.0137]],

         [[0.0159]]],


        [[[0.0149]],

         [[0.0156]],

         [[0.0152]],

         ...,

         [[0.0169]],

         [[0.0137]],

         [[0.0141]]],


        [[[0.0128]],

         [[0.0167]],

         [[0.0141]],

         ...,

         [[0.0172]],

         [[0.0185]],

         [[0.0154]]],


        ...,


        [[[0.0137]],

         [[0.0135]],

         [[0.0145]],

         ...,

         [[0.0145]],

         [[0.0145]],

         [[0.0123]]],


        [[[0.0133]],

         [[0.0133]],

         [[0.0147]],

         ...,

         [[0.0145]],

         [[0.0145]],

         [[0.0123]]],


        [[[0.0137]],

         [[0.0137]],

         [[0.0175]],

         ...,

         [[0.0179]],

         [[0.0179]],

         [[0.0154]]]])


In [43]:
class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []

        # Iterate over batches
        for batch in range(batch_size):
            # Iterate over channels
            channel_L_r = []
            for channel in range(channels):
                # Extract the current batch's unfolded window for the current channel
                windows_horizontal = image_scaled[batch, channel].unfold(0, self.window_size, 1).unfold(1, self.window_size, 1)
                windows_horizontal = windows_horizontal.squeeze(0)

                # Perform operations independently for each window in the current channel
                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(windows_horizontal)
                min_pool_output = -max_pool(-windows_horizontal)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2

                # Accumulate the result for the current channel
                channel_L_r.append(L_r)

            # Stack results across channels for the current batch
            channel_L_r = torch.stack(channel_L_r, dim = 1)
            all_L_r.append(channel_L_r)

        # Stack results across batches
        all_L_r = torch.stack(all_L_r)

        return all_L_r
    

# Create an instance of the CustomPoolingLayer
custom_pooling_layer = CustomPoolingLayer(r=3, window_size=3)

# Assuming 'your_image_tensor' is your input tensor with shape [128, 1, 13, 13] (128 batches, 1 channel, 13x13 image)
image = torch.rand((1, 1, 4, 4), dtype=torch.float32)


# Get the result using the custom pooling layer
result = custom_pooling_layer(image)
print(result.shape)

torch.Size([2, 2, 3, 3])
torch.Size([1, 2, 1, 2, 1, 1])


In [62]:
class CustomPoolingLayer(nn.Module):
    def __init__(self, r=3, window_size=3):
        super(CustomPoolingLayer, self).__init__()
        self.r = r
        self.window_size = window_size

    def forward(self, image):
        image_scaled = image * 255
        batch_size, channels, image_height, image_width = image.size()

        # Initialize variables to accumulate results across batches
        all_L_r = []

        # Iterate over batches
        for batch in range(batch_size):
            # Iterate over channels
            channel_L_r = []
            for channel in range(channels):
                # Extract the current batch's unfolded window for the current channel
                windows_horizontal = image_scaled[batch, channel].unfold(0, self.window_size, 1).unfold(1, self.window_size, 1)

                # Perform operations independently for each window in the current channel
                max_pool = nn.MaxPool2d(kernel_size=self.window_size)
                max_pool_output = max_pool(windows_horizontal)
                min_pool_output = -max_pool(-windows_horizontal)

                nr = torch.ceil(max_pool_output / self.r) - torch.ceil(min_pool_output / self.r) - 1
                Mr = torch.sum(nr)
                Q_mr = nr / (self.window_size - self.r + 1)
                L_r = (Mr**2) * Q_mr / (Mr * Q_mr)**2
                windows_horizontal[1:2, 1:2, :, :] = L_r

                # Accumulate the result for the current channel
                channel_L_r.append(L_r)
                print(channel_L_r)

            # Stack results along the channel dimension
            channel_L_r = torch.stack(channel_L_r, dim=1)
            all_L_r.append(channel_L_r)

        # Stack results along the batch dimension
        all_L_r = torch.stack(all_L_r, dim=0)

        return all_L_r

# Example usage
custom_pooling = CustomPoolingLayer(r=3, window_size=3)
input_data = torch.rand(1, 3, 4, 4)
output = custom_pooling(input_data)


[tensor([[[[0.0159]],

         [[0.0125]]],


        [[[0.0125]],

         [[0.0125]]]])]
[tensor([[[[0.0159]],

         [[0.0125]]],


        [[[0.0125]],

         [[0.0125]]]]), tensor([[[[0.0132]],

         [[0.0132]]],


        [[[0.0128]],

         [[0.0128]]]])]
[tensor([[[[0.0159]],

         [[0.0125]]],


        [[[0.0125]],

         [[0.0125]]]]), tensor([[[[0.0132]],

         [[0.0132]]],


        [[[0.0128]],

         [[0.0128]]]]), tensor([[[[0.0127]],

         [[0.0154]]],


        [[[0.0135]],

         [[0.0154]]]])]
